In [49]:
from transformers import pipeline, set_seed, GPT2Tokenizer, GPT2LMHeadModel
from torch import tensor, numel
import torch
import pandas as pd
from bertviz import model_view

set_seed(42)

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [9]:
phrase = 'My friend was right about this class. It is so fun!'
encoded_phrase = tokenizer(phrase, return_tensors='pt')
encoded_phrase

{'input_ids': tensor([[3666, 1545,  373,  826,  546,  428, 1398,   13,  632,  318,  523, 1257,
            0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [10]:
response = model(**encoded_phrase, output_attentions=True, output_hidden_states=True)
len(response.attentions)

12

In [11]:
response.attentions[-1].shape # from the final decoder

torch.Size([1, 12, 13, 13])

In [12]:
encoded_phrase['input_ids'].shape

torch.Size([1, 13])

In [13]:
tokens = tokenizer.convert_ids_to_tokens(encoded_phrase['input_ids'][0])
tokens

['My',
 'Ġfriend',
 'Ġwas',
 'Ġright',
 'Ġabout',
 'Ġthis',
 'Ġclass',
 '.',
 'ĠIt',
 'Ġis',
 'Ġso',
 'Ġfun',
 '!']

In [45]:
arr = response.attentions[9][0][0]
arr.shape

torch.Size([13, 13])

In [47]:
n_digits = 3
attention_df = pd.DataFrame((torch.round(arr*10**n_digits)/(10**n_digits)).detach()).applymap(float)
attention_df.columns = tokens
attention_df.index = tokens
attention_df

,My,Ġfriend,Ġwas,Ġright,Ġabout,Ġthis,Ġclass,.,ĠIt,Ġis,Ġso,Ġfun,!
My,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġfriend,0.968,0.032,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġwas,0.824,0.145,0.031,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġright,0.979,0.008,0.007,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġabout,0.979,0.008,0.004,0.005,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġthis,0.924,0.031,0.007,0.006,0.016,0.016,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Ġclass,0.946,0.005,0.001,0.001,0.001,0.002,0.044,0.000,0.000,0.000,0.000,0.000,0.000
.,0.691,0.013,0.003,0.003,0.002,0.006,0.269,0.013,0.000,0.000,0.000,0.000,0.000
ĠIt,0.318,0.003,0.003,0.003,0.006,0.018,0.599,0.018,0.032,0.000,0.000,0.000,0.000
Ġis,0.331,0.006,0.002,0.002,0.003,0.018,0.533,0.013,0.062,0.030,0.000,0.000,0.000


In [50]:
model_view(response.attentions, tokens)

<IPython.core.display.Javascript object>

In [51]:
response.hidden_states[-1].shape

torch.Size([1, 13, 768])

In [52]:
response.logits.shape

torch.Size([1, 13, 50257])

In [53]:
pd.DataFrame(
    zip(tokens, tokenizer.convert_ids_to_tokens(response.logits.argmax(2)[0])),
    columns=['Sequence up until', 'Next token with highest probability']
)

,Sequence up until,Next token with highest probability
0,My,Ċ
1,Ġfriend,","
2,Ġwas,Ġa
3,Ġright,.
4,Ġabout,Ġthat
5,Ġthis,.
6,Ġclass,.
7,.,ĠI
8,ĠIt,'s
9,Ġis,Ġa


In [54]:
generator = pipeline('text-generation', model='gpt2')

In [62]:
generator(phrase, max_length=20, num_return_sequences=1, do_sample=False)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My friend was right about this class. It is so fun! I love it! I love the'}]

In [56]:
generator(phrase, max_length=20, num_return_sequences=1, do_sample=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My friend was right about this class. It is so fun! The kids love every minute of it'}]